In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from requests_html import HTML

________________________________________

In [2]:
url = "https://soundcloud.com/discover"

In [3]:
browser = webdriver.Chrome()
browser.get(url)

In [4]:
soup = BeautifulSoup(browser.page_source)

In [5]:
tracks = []
playlists = []
users = []

# Lưu link profile của producer
urls_user = []

In [6]:
# nên check lại bởi vì vài lần chưa scroll hết :<
def scroll_down():
    # Loop 2 lần cho chắc ăn (có thể cho 3 lần)
    for i in range(2):
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        lastCount = 0
        match = False
        
        while(match == False):
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage: 
                match = True
            else: 
                lastCount = lenOfPage

In [7]:
# nên check lại bởi vì vài lần chưa scroll hết :<
def scroll_up():
    # Loop 2 lần cho chắc ăn (có thể cho 3 lần)
    for i in range(2):
        lenOfPage = browser.execute_script("window.scrollTo(document.body.scrollHeight, 0);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        lastCount = 0
        match = False
        
        while(match == False):
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(document.body.scrollHeight,0);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage: 
                match = True
            else: 
                lastCount = lenOfPage

In [8]:
def list_to_tsv_csv(listItem, outputFile):
    df = pd.DataFrame(listItem)
    df.to_csv(outputFile, sep='\t')

In [9]:
# Đồng ý cookies
cookies = browser.find_element(By.XPATH,"//button[@id='onetrust-accept-btn-handler']")
cookies.click()

In [10]:
# bấm nút > đến hết
soup = BeautifulSoup(browser.page_source)
genres = soup.find_all("li", "mixedModularHome__item")

for i in range(len(genres)):
    lists = genres[i].find_all("div", "tileGallery__sliderPanelSlide")
    j = 4
    while True:
        if j > len(lists):
            break
        browser.find_elements(By.XPATH,"//button[contains(.,'forward')]")[i].click()
        j += 4

In [11]:
# lấy hết tất cả playlists
soup = BeautifulSoup(browser.page_source)  
genres = soup.find_all("li", "mixedModularHome__item")
links = []

for i in range(len(genres)):
    lists = genres[i].find_all("div", "tileGallery__sliderPanelSlide")
    for pl in lists:
        title = pl.find("a", "playableTile__heading").get_text().strip()
        playlists.append(title)
        link = pl.find("a", "playableTile__artworkLink")["href"]
        links.append("https://soundcloud.com" + link)

In [12]:
def doSth(): 
    scroll_down()
    
    soup = BeautifulSoup(browser.page_source)

    # Trích xuất link profile của các producer có mặt trong 1 tab genre
    time.sleep(3)
    search = soup.find_all("li", "systemPlaylistTrackList__item")
    # time.sleep(3)
    for producer in search:
        user = producer.find("a", "trackItem__username").get_text()
        url = producer.find("a", "trackItem__username").get('href')
        track = producer.find("a", "trackItem__trackTitle").get_text()
        if user not in users:
            users.append(user)
            urls_user.append(url)
        if track not in tracks:
            tracks.append(track)

**Lấy tất cả tracks của một user**

In [ ]:
def get_all_tracks():
    scroll_down()
    time.sleep(2)
    soup = BeautifulSoup(browser.page_source)
    
    songs = soup.find_all("li", "soundList__item")
    for track in songs:
        title = track.find("a", "soundTitle__title").get_text().strip()
        # kiểm tra có bị trùng
        if title not in tracks:
            tracks.append(title)

**Lấy tất cả playlists của một user**

In [ ]:
def get_all_playlists():
    time.sleep(3)
    tab_playlists = browser.find_element(By.XPATH,'//a[contains(@href,"/sets")]')
    tab_playlists.click()
    
    time.sleep(3)
    scroll_down()
    soup = BeautifulSoup(browser.page_source)
    
    posts = soup.find_all("li", "soundList__item")
    for playlist in posts:
        title = playlist.find("a", "soundTitle__title")
        if title is None:
            break
        playlists.append(title.get_text().strip())

In [ ]:
def parse_tracks_per_genre():
    for url in urls_user:   
        browser.execute_script('window.open("{link}");'.format(link = url));
        browser.switch_to.window(browser.window_handles[1])
        get_all_tracks()
        time.sleep(2)
        get_all_playlists()
        time.sleep(2)
        browser.close()
        browser.switch_to.window(browser.window_handles[0])

In [ ]:
parse_tracks_per_genre()

In [13]:
def parse_per_genre():
    for url in links:   
        browser.execute_script('window.open("{link}");'.format(link = url));
        browser.switch_to.window(browser.window_handles[1])
        doSth()
        time.sleep(2)
        browser.close()
        browser.switch_to.window(browser.window_handles[0])

**Lưu data vào file tsv**

In [ ]:
outputFile = "playlists.tsv"
list_to_tsv_csv(playlists, outputFile)
print(len(playlists))

In [ ]:
parse_per_genre()

In [ ]:
outputFile = "user.tsv"
list_to_tsv_csv(users, outputFile)
outputFile = "track.tsv"
list_to_tsv_csv(tracks, outputFile)